### Helpful links:

### Point Cloud Processing:

- https://github.com/szenergy/awesome-lidar
- https://github.com/PointCloudLibrary/pcl
- https://github.com/daavoo/pyntcloud
- https://github.com/isl-org/Open3D
- https://pdal.io/stages/filters.html
- https://www.danielgm.net/cc/

### Visualization:

- https://deck.gl/
- https://kepler.gl/
- https://plas.io/

## To Do:

1. Think about folder structure and file names for processing many tiles consecutively 
2. Normalize LiDAR point clouds and save them as .npy format (scaling must be relative to the height of the tallest building)
3. Enrich point clouds with street view/aerial data

In [ ]:
from shapely.geometry import box
from typing import List

import plotly.graph_objects as go
import geopandas as gpd
import numpy as np

import os
import pdal
import shapely
import json
import laspy
import random

In [ ]:
%cd ..
%cd assets
assert os.getcwd() == '/Users/kevin/Projects/CS224W_LIDAR/assets', "You are not in the assets DIR"

In [ ]:
# Configuration
POINT_COUNT_THRESHOLD = 2000

In [ ]:
def visualize_3d_array(point_cloud_array: np.ndarray=None, file_name_list: List=None, example_ID=None):
    
    x = point_cloud_array[example_ID, :, 0].flatten()
    y = point_cloud_array[example_ID, :, 1].flatten()
    z = point_cloud_array[example_ID, :, 2].flatten()

    scatter = go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                         marker=dict(size = 3, color = z, colorscale = 'Viridis'))
    layout = go.Layout(title = f'Visualization of {file_name_list[example_ID]}')
    fig = go.Figure(data = [scatter], layout = layout)
    fig.show()

def _convert_numpy_to_las(x: np.ndarray=None, header=None):
    
    outfile = laspy.LasData(header)
    outfile.x = x[:,0]
    outfile.y = x[:,1]
    outfile.z = x[:,2]
    outfile.intensity = x[:,3]
    outfile.raw_classification = x[:,4]
    outfile.scan_angle_rank = x[:,5]
    
    return outfile

def _sample_random_points(x: np.ndarray=None, subset_size: int=None):
    
    rng = np.random.default_rng()
    lidar_subset = rng.choice(a=x, size=subset_size, replace=False, axis=0)
    
    return lidar_subset

def _convert_las_to_numpy(las_data = None):
    
    lidar_numpy = np.array((las_data.x, 
                             las_data.y, 
                             las_data.z, 
                             las_data.intensity, 
                             las_data.raw_classification, 
                             las_data.scan_angle_rank)).transpose()
    
    return lidar_numpy
        
def subsample_las(original_las_data_filepath: str=None, subset_size: int=1000000):

    org_las_data = laspy.read(original_las_data_filepath)
    # Set meta data for new LAS file based on settings from original LAS file
    hdr = org_las_data.header
    hdr.point_count = 0
    
    lidar_ndarray = _convert_las_to_numpy(las_data = org_las_data)
    print(f"SHAPE of LIDAR: {lidar_ndarray.shape}")
    
    lidar_subset = _sample_random_points(x=lidar_ndarray, subset_size=subset_size)
    print(f"SHAPE of LIDAR_SUBSET: {lidar_subset.shape}")
    
    outfile = _convert_numpy_to_las(lidar_subset, hdr)
    output_filepath = original_las_data_filepath[:-4] + "_SUBSET.las"
    
    print(f"Saving subsampled LAS file to: {output_filepath}")
    outfile.write(output_filepath)
    
    return outfile

def create_tile_bounding_box(original_las_data_filepath: str=None):
    
    las_data = laspy.read(original_las_data_filepath)
    min_x, min_y, min_z, max_x, max_y, max_z = [*las_data.header.min, *las_data.header.max]
    return box(minx=min_x, miny=min_y, maxx=max_x, maxy=max_y)

In [ ]:
#test = subsample_las("SP3278_P_11321_20171123_20171123.las")
#test

In [ ]:
osm_footprints = gpd.read_file("coventry_building_footprints.geojson")

# Buffer polygons a bit to capture the building footprint better
osm_footprints["geometry"] = osm_footprints["geometry"].buffer(1)

# Reproject OSM footprints to EPSG:27700, if necessary
#osm_footprints = osm_footprints.to_crs("EPSG:27700")
#osm_footprints.to_file("coventry_building_footprints.geojson", driver='GeoJSON')

footprint_list = osm_footprints['geometry'].tolist()

# Select only polygons which are within the LiDAR tile and save their WKT string
lidar_bounding_box = create_tile_bounding_box("SP3278_P_11321_20171123_20171123.las")
polys = [elem.wkt for elem in footprint_list if isinstance(elem, shapely.geometry.polygon.Polygon) and lidar_bounding_box.contains(elem.centroid)]
print(f"Number of relevant polygons: {len(polys)}")

In [ ]:
# Sample a subset of the footprint polygons for debugging purposes
polys = random.sample(polys, 50)

for idx, polygon in enumerate(polys):
    
    pipeline_definition = {

        'pipeline': [

        "SP3278_P_11321_20171123_20171123.las",

        {
            "type":"filters.crop",
            "polygon":polygon
        },

        {
            "type":"writers.las",
            "filename":f"cropped_{idx}.las"
        }

        ]
    }

    pipeline = pdal.Pipeline(json.dumps(pipeline_definition))

    pipeline.execute()

In [ ]:
point_cloud_examples = []
point_cloud_filenames = []

for file_path in os.listdir(os.getcwd()):
    
    # All PDAL-based output files start with "cropped"
    if file_path.startswith("cropped"):
    
        las_point_cloud = laspy.read(file_path)
        point_count = len(las_point_cloud.points)

        if point_count < POINT_COUNT_THRESHOLD:
            os.remove(file_path)

        else:

            numpy_point_cloud = _convert_las_to_numpy(las_point_cloud)
            numpy_point_cloud = _sample_random_points(x=numpy_point_cloud, subset_size=POINT_COUNT_THRESHOLD)
            numpy_point_cloud = numpy_point_cloud[np.newaxis, ...]

            point_cloud_examples.append(numpy_point_cloud)
            point_cloud_filenames.append(file_path)

point_cloud_examples = np.concatenate(point_cloud_examples, axis=0)
point_cloud_examples.shape

In [ ]:
visualize_3d_array(point_cloud_examples, point_cloud_filenames, example_ID=3)